<a href="https://colab.research.google.com/github/Castaest/Data_Analysis_School/blob/main/ABD_%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D1%8C_AAB_ipynb%22%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Установка PySpark

In [1]:
!apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [802 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [108 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,087 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [848 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [969 kB]
Get:13 http://archive.ubuntu.com/ubuntu

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [3]:
!wget -q https://downloads.apache.org/spark/spark-3.4.0/spark-3.4.0-bin-hadoop3.tgz

In [4]:
!tar -xvf spark-3.4.0-bin-hadoop3.tgz > /dev/null

In [5]:
!pip install -q findspark

In [6]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.0-bin-hadoop3"

In [7]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.master("local[*]").getOrCreate()

## Загружаем данные из файла

In [8]:
df = spark.read.options(header='True', inferSchema='True').csv('/content/iris.CSV')

In [9]:
df.dtypes

[('sepal_length', 'double'),
 ('sepal_width', 'double'),
 ('petal_length', 'double'),
 ('petal_width', 'double'),
 ('variety', 'string'),
 ('variety_num', 'int')]

In [10]:
df.show(5)

+------------+-----------+------------+-----------+-------+-----------+
|sepal_length|sepal_width|petal_length|petal_width|variety|variety_num|
+------------+-----------+------------+-----------+-------+-----------+
|         5.1|        3.5|         1.4|        0.2| Setosa|          0|
|         4.9|        3.0|         1.4|        0.2| Setosa|          0|
|         4.7|        3.2|         1.3|        0.2| Setosa|          0|
|         4.6|        3.1|         1.5|        0.2| Setosa|          0|
|         5.0|        3.6|         1.4|        0.2| Setosa|          0|
+------------+-----------+------------+-----------+-------+-----------+
only showing top 5 rows



In [12]:
# проверка отсутствия пропусков
df.describe().show()

+-------+------------------+-------------------+------------------+------------------+---------+------------------+
|summary|      sepal_length|        sepal_width|      petal_length|       petal_width|  variety|       variety_num|
+-------+------------------+-------------------+------------------+------------------+---------+------------------+
|  count|               150|                150|               150|               150|      150|               150|
|   mean| 5.843333333333335|  3.057333333333334|3.7580000000000027| 1.199333333333334|     null|               1.0|
| stddev|0.8280661279778637|0.43586628493669793|1.7652982332594662|0.7622376689603467|     null|0.8192319205190406|
|    min|               4.3|                2.0|               1.0|               0.1|   Setosa|                 0|
|    max|               7.9|                4.4|               6.9|               2.5|Virginica|                 2|
+-------+------------------+-------------------+------------------+-----

In [15]:
# собираем все признаки в единый вектор
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['sepal_length', 'sepal_width', 'petal_length', 'petal_width'], outputCol='Features')
df_features = assembler.transform(df)
df_features.show(5)

+------------+-----------+------------+-----------+-------+-----------+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|variety|variety_num|         Features|
+------------+-----------+------------+-----------+-------+-----------+-----------------+
|         5.1|        3.5|         1.4|        0.2| Setosa|          0|[5.1,3.5,1.4,0.2]|
|         4.9|        3.0|         1.4|        0.2| Setosa|          0|[4.9,3.0,1.4,0.2]|
|         4.7|        3.2|         1.3|        0.2| Setosa|          0|[4.7,3.2,1.3,0.2]|
|         4.6|        3.1|         1.5|        0.2| Setosa|          0|[4.6,3.1,1.5,0.2]|
|         5.0|        3.6|         1.4|        0.2| Setosa|          0|[5.0,3.6,1.4,0.2]|
+------------+-----------+------------+-----------+-------+-----------+-----------------+
only showing top 5 rows



In [17]:
# если бы этапов обработки данных было бы много, то было бы выгодно использовать Pipeline
from pyspark.ml import Pipeline
pipeline = Pipeline(stages =
[
  VectorAssembler(inputCols=['sepal_length', 'sepal_width', 'petal_length', 'petal_width'], outputCol='Features')
]
)
pipelineTrained = pipeline.fit(df)
df_m = pipelineTrained.transform(df)
df_m.show(5)

+------------+-----------+------------+-----------+-------+-----------+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|variety|variety_num|         Features|
+------------+-----------+------------+-----------+-------+-----------+-----------------+
|         5.1|        3.5|         1.4|        0.2| Setosa|          0|[5.1,3.5,1.4,0.2]|
|         4.9|        3.0|         1.4|        0.2| Setosa|          0|[4.9,3.0,1.4,0.2]|
|         4.7|        3.2|         1.3|        0.2| Setosa|          0|[4.7,3.2,1.3,0.2]|
|         4.6|        3.1|         1.5|        0.2| Setosa|          0|[4.6,3.1,1.5,0.2]|
|         5.0|        3.6|         1.4|        0.2| Setosa|          0|[5.0,3.6,1.4,0.2]|
+------------+-----------+------------+-----------+-------+-----------+-----------------+
only showing top 5 rows



In [22]:
# разделение на обучающую и тестовую выборку
train, test = df_m.randomSplit([0.8, 0.2], seed = 42)
train.show(5)

+------------+-----------+------------+-----------+-------+-----------+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|variety|variety_num|         Features|
+------------+-----------+------------+-----------+-------+-----------+-----------------+
|         4.3|        3.0|         1.1|        0.1| Setosa|          0|[4.3,3.0,1.1,0.1]|
|         4.4|        2.9|         1.4|        0.2| Setosa|          0|[4.4,2.9,1.4,0.2]|
|         4.4|        3.2|         1.3|        0.2| Setosa|          0|[4.4,3.2,1.3,0.2]|
|         4.5|        2.3|         1.3|        0.3| Setosa|          0|[4.5,2.3,1.3,0.3]|
|         4.6|        3.1|         1.5|        0.2| Setosa|          0|[4.6,3.1,1.5,0.2]|
+------------+-----------+------------+-----------+-------+-----------+-----------------+
only showing top 5 rows



In [23]:
# обучение модели Random Forest
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [35]:
rf = RandomForestClassifier(featuresCol = 'Features', labelCol = 'variety_num', numTrees = 30, maxDepth = 3)
rfModel = rf.fit(train)

In [36]:
train_tr_res = rfModel.transform(train)
test_tr_res = rfModel.transform(test)

In [37]:
evaluator = MulticlassClassificationEvaluator()
evaluator.setLabelCol('variety_num')
evaluator.setPredictionCol('prediction');

In [46]:
print('Оценка качества модели на обучающей выборке: ', round(evaluator.evaluate(train_tr_res), 4))
print('Оценка качества модели на тестовой выборке: ', round(evaluator.evaluate(test_tr_res), 4))

Оценка качества модели на обучающей выборке:  0.9762
Оценка качества модели на тестовой выборке:  1.0


###Датасет очень простой, поэтому Random Forest показывает идеальный результат на тестовой выборке

In [47]:
train_tr_res.show()

+------------+-----------+------------+-----------+----------+-----------+-----------------+--------------------+--------------------+----------+
|sepal_length|sepal_width|petal_length|petal_width|   variety|variety_num|         Features|       rawPrediction|         probability|prediction|
+------------+-----------+------------+-----------+----------+-----------+-----------------+--------------------+--------------------+----------+
|         4.3|        3.0|         1.1|        0.1|    Setosa|          0|[4.3,3.0,1.1,0.1]|[29.9259259259259...|[0.99753086419753...|       0.0|
|         4.4|        2.9|         1.4|        0.2|    Setosa|          0|[4.4,2.9,1.4,0.2]|[29.9259259259259...|[0.99753086419753...|       0.0|
|         4.4|        3.2|         1.3|        0.2|    Setosa|          0|[4.4,3.2,1.3,0.2]|[29.9259259259259...|[0.99753086419753...|       0.0|
|         4.5|        2.3|         1.3|        0.3|    Setosa|          0|[4.5,2.3,1.3,0.3]|[29.0509259259259...|[0.96836419